In [ ]:
!pip install datasets


In [ ]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df = pd.read_csv('/content/Preprocessed Fake Reviews Detection Dataset.csv')
df.head()

,Unnamed: 0,category,rating,label,text_
0,0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort i love veri pretti
1,1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin i 've mine coupl year
2,2,Home_and_Kitchen_5,5.0,CG,thi pillow save back i love look feel pillow
3,3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price i
4,4,Home_and_Kitchen_5,5.0,CG,veri nice set good qualiti we set two month


In [ ]:
# Randomly sample 360 rows (set random_state for reproducibility)
df = df.sample(n=200, random_state=42)
df.shape

(200, 4)

In [ ]:
df

,category,rating,label,text_
34814,Toys_and_Games_5,4.0,1,thi box includ tattoo boy girl great most kit ...
9753,Electronics_5,5.0,1,thi awesom radio i two show friend order one g...
19847,Pet_Supplies_5,1.0,1,did work either dog they wiggl right everi time
7379,Sports_and_Outdoors_5,4.0,0,we normal play nbsp data-hook= '' product-link...
22584,Pet_Supplies_5,5.0,1,we use long time puppi one blue later red adul...
...,...,...,...,...
30307,Books_5,4.0,1,i found interest inform it excel first hand ac...
1451,Home_and_Kitchen_5,5.0,1,thi organ fit nice bottom closet also look gre...
34996,Toys_and_Games_5,5.0,1,thi great famili game fun parti game we receiv...
10427,Electronics_5,5.0,1,switch new balanc runner what differ it take 1...


In [ ]:
df = df.drop(columns=["Unnamed: 0"])
df["label"] = df["label"].map({"CG": 0, "OR": 1})
df.head()

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text_"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
train_texts = [str(text) for text in train_texts]
test_texts = [str(text) for text in test_texts]

# Convert the lists to Hugging Face Dataset objects
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset  = Dataset.from_dict({"text": test_texts, "label": test_labels})

In [ ]:
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset  = Dataset.from_dict({"text": test_texts, "label": test_labels})

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset  = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

<ipython-input-31-ded50a0afaca>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.702415


In [ ]:
def predict_review(review):
    # Tokenize the input review
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True, max_length=512)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    # Get the predicted label (0 for Fake, 1 for Real)
    prediction = torch.argmax(logits, dim=1).item()
    return "Real" if prediction == 1 else "Fake"

In [ ]:
sample_review = "This product is excellent and works perfectly!"
print(f"Review: {sample_review}\nPrediction: {predict_review(sample_review)}")

In [ ]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")